In [1]:
%jsroot on

In [2]:
TString fedrapath(gSystem->Getenv("FEDRA_ROOT"));
gROOT->ProcessLine((TString(".x ")+fedrapath+TString("/macros/rootlogon.C")).Data())

Load FEDRA libs
(long) 0


# Open the file and getting the TTree

The TTree is named "vtx"

## Vertex Tree Data format

The Tree contains the following information
Scalar branches (i.e. one number for vertex):

* n: vertex molteplicity (accessed with N());

* vx,vy,vz: coordinates of reconstructed vertex (accessed with VX(), VY(), VZ());

* vCOV: covariance matrix of vertex position (stored as TMatrixD)

* meanvx,meanvy,meanvz: mean point of starting tracks (accessed with X(), Y(),Z());

* maxaperture; maximum angle between tracks in the vertex (computed with MaxAperture ());

* probability: probability of vertex from fit (accessed with V()->prob());

* flag: a flag providing information about vertex topology (accessed with Flag()):
        flag 0: neutral (tracks' starts attached only
        flag 1: charge (tracks' ends&starts attached
        flag 2: back neutral (tracks' ends attached only

    The following flags are similar to the previous one, but they add the information that vertex has common tracks to at least another vertex (if LinkedVertexes () has been called)
        flag 3: neutral, linked (has common tracks with other vertex, only if LinkedVertexes () has been called)
        flag 4: charge, linked
        flag 5: back neutral, linked
        flag -10: discarded vertices (all tracks were associated to higher rank vertices).

In [3]:
TString filepath(
//"/eos/user/f/falicant/handson_FEDRA/b000021/trackfiles/rootfiles/47_41/cell_9_9/vertextree.root");    
"/eos/experiment/sndlhc/emulsionData/2022/emureco_Napoli/RUN3/b000044/trackfiles/rootfiles/57_1/cell_7_4/vertextree.root");
TFile *vertexfile = TFile::Open(filepath);
TTree *vertextree = (TTree*) vertexfile->Get("vtx")


(TTree *) 0x7f505d991c90


In [4]:
using namespace ROOT;

In [5]:
RDataFrame df(*vertextree); //loading a dataframe with tracks info

In [6]:
//getting n, flag and probability
auto hn = df.Histo1D<int>("n"); //like TTree->Draw(), RDataFrame can infer range from data
auto hflag = df.Histo1D<int>("flag");
auto hprob = df.Histo1D({"hprob","Probability",110,0.,1.1},"probability"); //or we can provide it


In [7]:
//draw histograms
TCanvas *cprob= new TCanvas();
hprob->DrawClone();
cprob->Draw();

In [8]:
//draw histograms
TCanvas *cn= new TCanvas();
hn->DrawClone();
cn->Draw();

In [9]:
TCanvas *cflag= new TCanvas();
hflag->DrawClone();
cflag->Draw();

Array branches (i.e. track properties):

* nseg[itrk]: number of found segments for each track (accessed with track->N());
* npl[itrk]: number of expeceted segments for each track (accessed with track->Npl());
* incoming[itrk]: tell us if track ends or start at vertex (accessed with GetVTa (itrk)->ZPos()). 1-track start, 0-track end connect to the vertex
* nholes[itrk]; number of holes for each track (accessed with track->N0());
* maxgap[itrk]: maximum gap between segments of each track (accessed with track->CheckMaxGap());
* impactparameter[itrk]: impact parameter of track with respect to vertex (accessed with vertex->GetVTa(itrk)->Imp());

Arrays of FEDRA objects (as from linked_tracks.root format)

* t.: TClonesArray of EdbSegP with tracks information;
* s: TClonesArray of EdbSegP with segments information for each track;
* sf: TClonesArray of EdbSegP with fitted segments information for each track;

The following branches are to be used only in vertices from MC simulations, no sense in real data:

 * MCEventID[itrk]: true MCEventID of each track (accessed with track->MCEvt());
 * MCTrackID[itrk]: true MCTrackID of each track (accessed with track->MCTrack());
 * MCTrackPdgCode[itrk]: true MCTrack PdgCode of each track (accessed with track->Vid(0));
 * MCMotherID[itrk]: true MCMotherID of each track (accessed with track->Aid(0));

In [10]:
auto mydisplay = df.Filter("flag==4 && n>4").Display({"vID","vz","n","t.eZ","incoming"},10);
mydisplay->Print();

+-------+-------+-----------+---+-----------+----------+
| Row   | vID   | vz        | n | t.eZ      | incoming | 
+-------+-------+-----------+---+-----------+----------+
| 45445 | 45445 | -53446.8f | 5 | -64450.6f | 0        | 
|       |       |           |   | -60381.6f | 0        | 
|       |       |           |   | -61700.8f | 0        | 
|       |       |           |   | -51179.7f | 1        | 
|       |       |           |   | -64450.6f | 0        | 
+-------+-------+-----------+---+-----------+----------+
| 45617 | 45617 | -13198.1f | 5 | -73797.6f | 0        | 
|       |       |           |   | -22251.3f | 0        | 
|       |       |           |   | -11794.7f | 1        | 
|       |       |           |   | -51179.7f | 0        | 
|       |       |           |   | -73797.6f | 0        | 
+-------+-------+-----------+---+-----------+----------+
| 46246 | 46246 | -38556.4f | 5 | -60381.6f | 0        | 
|       |       |           |   | -48546.7f | 0        | 
|       |       | 

In [11]:
auto mydisplay = df.Filter("flag==3 && n>4").Display({"vID","vz","n","t.eZ","incoming"},10);
mydisplay->Print();

+-------+-------+-----------+---+-----------+----------+
| Row   | vID   | vz        | n | t.eZ      | incoming | 
+-------+-------+-----------+---+-----------+----------+
| 62371 | 62371 | -57308.2f | 5 | -56354.7f | 1        | 
|       |       |           |   | -56354.7f | 1        | 
|       |       |           |   | -56354.7f | 1        | 
|       |       |           |   | -56354.7f | 1        | 
|       |       |           |   | -56354.7f | 1        | 
+-------+-------+-----------+---+-----------+----------+
| 62377 | 62377 | -74960.5f | 5 | -73797.6f | 1        | 
|       |       |           |   | -73797.6f | 1        | 
|       |       |           |   | -73797.6f | 1        | 
|       |       |           |   | -72446.1f | 1        | 
|       |       |           |   | -73797.6f | 1        | 
+-------+-------+-----------+---+-----------+----------+
| 62378 | 62378 | -35392.0f | 6 | -32862.8f | 1        | 
|       |       |           |   | -32862.8f | 1        | 
|       |       | 